# Test instantiating PypIt parameter sets

In [1]:
# import
import os
from configobj import ConfigObj
from pypit.par import pypitpar

## General usage

In [2]:
# To get the default parameters, declare the parameters set
# without arguments
p = pypitpar.ProcessImagesPar()
# Use print() to get a short-form representation
print('Print output:')
print(p)
# Use the info() method to get a long-form representation
print('.info() output:')
p.info()
# Both of these can go haywire for complex constructions of
# parameter sets (sets within sets) but are well formatted
# in the simplest cases

# Use the to_config() method to output to a *.cfg file with
# comments for each item.  You have to provide a section
# name in most cases.
p.to_config('test_process.cfg', section_name='process')
# You can also use the to_config() method to just get the
# list of strings that are then written to the file.  This
# is useful when you want to reconstruct a ConfigObj.
p_cfgobj = ConfigObj(p.to_config(cfg_file=None, section_name='process'))
print(p_cfgobj)

Print output:
Parameter     Value       Default     Type        Callable
----------------------------------------------------------
overscan      savgol      savgol      str         False   
overscan_par  5, 65       5, 65       int, list   False   
match         -1          -1          int, float  False   
combine       weightmean  weightmean  str         False   
satpix        reject      reject      str         False   
sigrej        20.0        20.0        int, float  False   
n_lohi        0, 0        0, 0        list        False   
sig_lohi      3.0, 3.0    3.0, 3.0    list        False   
replace       maxnonsat   maxnonsat   str         False   
lamaxiter     1           1           int         False   
grow          1.5         1.5         int, float  False   
rmcompact     True        True        bool        False   
sigclip       5.0         5.0         int, float  False   
sigfrac       0.3         0.3         int, float  False   
objlim        5.0         5.0         int,

## Demo of parameter defaults

The following just shows the results of the default parameter sets to show what parameters are defined and where.


### Reduction parameter set

The reduction parameter set provides some global parameters for the data reduction.

In [3]:
pypitpar.ReducePar()

Parameter     Value    Default  Type        Callable
----------------------------------------------------
spectrograph  None     None     str         False   
pipeline      None     None     str         False   
detnum        None     None     list        False   
sortroot      None     None     str         False   
calwin        0        0        int, float  False   
scidir        Science  Science  str         False   
qadir         QA       QA       str         False   

### Frame groups

Parameters used for each frame type (bias, pixelflat, etc) are abstracted to a single "Frame-Group" parameter set.  The default is 'bias'; valid frame types are shown using the static method `valid_frame_types` (see below).  The parameters used to process the images in the frame group is a subset of `FrameGroupPar`, called `ProcessImagesPar`.

In [4]:
pypitpar.ProcessImagesPar()

Parameter     Value       Default     Type        Callable
----------------------------------------------------------
overscan      savgol      savgol      str         False   
overscan_par  5, 65       5, 65       int, list   False   
match         -1          -1          int, float  False   
combine       weightmean  weightmean  str         False   
satpix        reject      reject      str         False   
sigrej        20.0        20.0        int, float  False   
n_lohi        0, 0        0, 0        list        False   
sig_lohi      3.0, 3.0    3.0, 3.0    list        False   
replace       maxnonsat   maxnonsat   str         False   
lamaxiter     1           1           int         False   
grow          1.5         1.5         int, float  False   
rmcompact     True        True        bool        False   
sigclip       5.0         5.0         int, float  False   
sigfrac       0.3         0.3         int, float  False   
objlim        5.0         5.0         int, float  False 

In [5]:
pypitpar.FrameGroupPar.valid_frame_types()

['bias', 'pixelflat', 'arc', 'pinhole', 'trace', 'standard', 'science', 'all']

In [6]:
pypitpar.FrameGroupPar()

Parameter  Value      Default    Type          Callable
-------------------------------------------------------
frametype  bias       bias       str           False   
useframe   bias       None       str           False   
number     0          0          int           False   
process    see below  see below  ParSet, dict  False   

process
Parameter     Value       Default     Type        Callable
----------------------------------------------------------
overscan      savgol      savgol      str         False   
overscan_par  5, 65       5, 65       int, list   False   
match         -1          -1          int, float  False   
combine       weightmean  weightmean  str         False   
satpix        reject      reject      str         False   
sigrej        20.0        20.0        int, float  False   
n_lohi        0, 0        0, 0        list        False   
sig_lohi      3.0, 3.0    3.0, 3.0    list        False   
replace       maxnonsat   maxnonsat   str         False   
lamaxi

### Calibrations parameter set

CalibrationsPar is a super set of other parameter groups that are needed for the calibration data.  It is built up of 10 other parameter sets, plus a few additional single parameters like where to save the master frames.  Six of these are instances of `FrameGroupPar` for the bias, arc, pixelflat, pinhole, trace and standard frames (see above).  The remaining four set how to peform the field flattening (`FlatFieldPar`), determine the wavelength solution (`WavelengthSolutionPar`), trace the spatial position of the slits along the dispersion axis (`TraceSlitsPar`), and trace the shift in the wavelength solution as a function of spatial position within a slit (`WaveTiltsPar`)

In [7]:
pypitpar.FlatFieldPar()

Parameter    Value      Default    Type       Callable
------------------------------------------------------
frame        pixelflat  pixelflat  str        False   
slitprofile  True       True       bool       False   
method       bspline    bspline    str        False   
params       20         20         int, list  False   
twodpca      0          0          int        False   

In [8]:
pypitpar.WavelengthSolutionPar()

Parameter  Value         Default       Type              Callable
-----------------------------------------------------------------
reference  arc           arc           str               False   
method     arclines      arclines      str               False   
lamps      None          None          list              False   
detection  6.0           6.0           int, float        False   
numsearch  20            20            int               False   
nfitpix    5             5             int               False   
IDpixels   None          None          int, list         False   
IDwaves    None          None          int, float, list  False   
medium     vacuum        vacuum        str               False   
frame      heliocentric  heliocentric  str               False   

In [9]:
pypitpar.TraceSlitsPar()

Parameter      Value             Default           Type        Callable
-----------------------------------------------------------------------
function       legendre          legendre          str         False   
polyorder      3                 3                 int         False   
medrep         0                 0                 int         False   
number         -1                -1                int         False   
trim           3, 3              3, 3              tuple       False   
maxgap         None              None              int         False   
maxshift       0.15              0.15              int, float  False   
pad            0                 0                 int         False   
sigdetect      20.0              20.0              int, float  False   
fracignore     0.01              0.01              float       False   
diffpolyorder  2                 2                 int         False   
single         []                []                list        F

In [10]:
pypitpar.WaveTiltsPar()

Parameter    Value     Default   Type                       Callable
--------------------------------------------------------------------
idsonly      False     False     bool                       False   
tracethresh  1000.0    1000.0    int, float, list, ndarray  False   
order        2         2         int                        False   
function     legendre  legendre  str                        False   
yorder       4         4         int                        False   
func2D       legendre  legendre  str                        False   
method       spca      spca      str                        False   
params       1, 1, 0   1, 1, 0   int, list                  False   

In [11]:
# Putting it all together, the CalibrationsPar parameter set looks like this:
pypitpar.CalibrationsPar()

Parameter       Value      Default    Type          Callable
------------------------------------------------------------
caldir          MF         MF         str           False   
masters         None       None       str           False   
setup           None       None       str           False   
trim            True       True       bool          False   
badpix          True       True       bool          False   
biasframe       see below  see below  ParSet, dict  False   
arcframe        see below  see below  ParSet, dict  False   
pixelflatframe  see below  see below  ParSet, dict  False   
pinholeframe    see below  see below  ParSet, dict  False   
traceframe      see below  see below  ParSet, dict  False   
standardframe   see below  see below  ParSet, dict  False   
flatfield       see below  see below  ParSet, dict  False   
wavelengths     see below  see below  ParSet, dict  False   
slits           see below  see below  ParSet, dict  False   
tilts           see belo

### Object tracing

Parameters guiding the tracing of objects in the slit:

In [12]:
pypitpar.TraceObjectsPar()

Parameter  Value     Default   Type        Callable
---------------------------------------------------
function   legendre  legendre  str         False   
order      2         2         int         False   
find       standard  standard  str         False   
nsmooth    3         3         int, float  False   
xedge      0.03      0.03      float       False   
method     pca       pca       str         False   
params     1, 0      1, 0      int, list   False   

### Object extraction

I provide an object that defines any manual extractions (a bit overkill):

In [13]:
pypitpar.ManualExtractionPar()

Parameter  Value  Default  Type  Callable
-----------------------------------------
frame      None   None     str   False   
params     None   None     list  False   

Then the `manual` parameter in the `ExtractObjectsPar` is a list of the `ManualExtractionPar` objects for all the manual extractions to perform.  The number of manual extractions is just the length of the list or 0 if the element is `None`.

In [14]:
p = pypitpar.ExtractObjectsPar()
print(p)  # The printing goes a bit wrong for 'manual'
nmanual = 0 if p['manual'] is None else len(p['manual'])
print('Manual extractions to perform: {0}'.format(nmanual))

Parameter   Value     Default   Type        Callable
----------------------------------------------------
pixelmap    None      None      str         False   
pixelwidth  2.5       2.5       int, float  False   
reuse       False     False     bool        False   
profile     gaussian  gaussian  str         False   
maxnumber   None      None      int         False   
manual      None      None      list        False   

Manual extractions to perform: 0


Finally, there are parameter sets that used for the sky subtraction, flexure correction, and flux calibration.

In [15]:
#6
pypitpar.SkySubtractionPar()

Parameter        Value  Default  Type        Callable
-----------------------------------------------------
bspline_spacing  0.6    0.6      int, float  False   
nodding          False  False    bool        False   

In [16]:
#3
pypitpar.FlexurePar()

Parameter  Value   Default  Type        Callable
------------------------------------------------
method     boxcar  boxcar   str         False   
maxshift   20      20       int, float  False   
spectrum   None    None     str         False   

In [17]:
#5
pypitpar.FluxCalibrationPar()

Parameter  Value  Default  Type  Callable
-----------------------------------------
nonlinear  False  False    bool  False   
sensfunc   None   None     str   False   

## Putting it all together

All of the above are expected to be parameter subsets that are passed to individual methods.  Sticking with the idea of having a single object that provides all the parameters needed for a run of `pypit`, the `PypitPar` object just collects all the above objects into a single parameter set.

In [18]:
pypitpar.PypitPar()

Parameter     Value      Default    Type          Callable
----------------------------------------------------------
rdx           see below  see below  ParSet, dict  False   
calibrations  see below  see below  ParSet, dict  False   
scienceframe  see below  see below  ParSet, dict  False   
objects       see below  see below  ParSet, dict  False   
extract       see below  see below  ParSet, dict  False   
skysubtract   None       None       ParSet, dict  False   
flexure       None       None       ParSet, dict  False   
fluxcalib     None       None       ParSet, dict  False   

rdx
Parameter     Value    Default  Type        Callable
----------------------------------------------------
spectrograph  None     None     str         False   
pipeline      None     None     str         False   
detnum        None     None     list        False   
sortroot      None     None     str         False   
calwin        0        0        int, float  False   
scidir        Science  Science  st

Like all the other pypit parameter sets, `PypitPar` has a `from_dict` method, but it also has a `from_cfg_file` that allows you to read and write the full parameter sets to a `*.cfg` config file.  However, for runs of pypit, we're most often going to be reading the configuration from a group of lines in the `*.pypit` file.  Here's a short demo of how to read and write the pypit parameters to a `*.cfg`; writing may be useful to see the full set of parameters along with the description of each parameter.

In [19]:
help(pypitpar.PypitPar.from_cfg_file)

Help on method from_cfg_file in module pypit.par.pypitpar:

from_cfg_file(cfg_file=None, merge_with=None, evaluate=True) method of builtins.type instance
    Construct the parameter set using a configuration file.
    
    Note that::
    
        default = PypitPar()
        nofile = PypitPar.from_cfg_file()
        assert default.data == nofile.data, 'This should always pass.'
    
    Args:
        cfg_file (:obj:`str`, optional):
            The name of the configuration file that defines the
            default parameters.  This can be used to load a pypit
            config file from a previous run that was constructed and
            output by pypit.  This has to contain the full set of
            parameters, not just the subset you want to change.  For
            the latter, use :arg:`merge_with` to provide one or more
            config files to merge with the defaults to construct the
            full parameter set.
        merge_with (:obj:`str`, :obj:`list`, optional):
  

You can get the default parameters and print the result like this.

In [20]:
# Write the defaults
p = pypitpar.PypitPar.from_cfg_file()
p.to_config('default.cfg')

You can then read it back in.  The `expand_spectrograph` option allows you to set all the spectrograph setting using the `spectrograph` keyword in the input config file (this is `KECK_LRISb` by default).  If you just want to use what's in the config file without finding the spectrograph configuration file (because those keywords are already in the configuration file in this example), you would set `expand_spectrograph=False`.

In [21]:
# Read them in
p = pypitpar.PypitPar.from_cfg_file('default.cfg')
p['rdx']['spectrograph'] is None

True

## User-level example

I import `PypitPar` as part of the `__init__.py` setup of the `pypit.par` module, meaning you can import it directly.  I expect this is how we'll use it in the rest of the code.

In [22]:
from pypit.par import PypitPar

Below are a few examples of how to setup a `PypitPar` instance.  In all these cases, you only have to specify the parameters that are different from the defaults.

### Using `PypitSetup`

The most common usage for setting up a `PypitPar` instance will be to use `PypitSetup`.  For example:

In [23]:
from pypit import pypitsetup
pypit_file = '../../pypit/tests/files/example_pypit_file.pypit'
setup = pypitsetup.PypitSetup.from_pypit_file(pypit_file)
par, spectrograph, fitstbl, setup_dict = setup.run()

[INFO]    :: Loading the reduction file
[INFO]    :: Found 2 raw data frames
[INFO]    :: Input file loaded successfully
[WARNING] :: UTC is not listed as a header keyword in file:
             /Users/westfall/Work/packages/PYPIT/pypit/tests/files/b1.fits.gz
[WARNING] :: BINNING keyword not in header. Setting to None
[INFO]    :: Successfully loaded headers for file:
             /Users/westfall/Work/packages/PYPIT/pypit/tests/files/b1.fits.gz
[WARNING] :: UTC is not listed as a header keyword in file:
             /Users/westfall/Work/packages/PYPIT/pypit/tests/files/b27.fits.gz
[WARNING] :: BINNING keyword not in header. Setting to None
[INFO]    :: Successfully loaded headers for file:
             /Users/westfall/Work/packages/PYPIT/pypit/tests/files/b27.fits.gz
[INFO]    :: Checking spectrograph settings for required header information
[INFO]    :: Headers loaded for 2 files successfully
[INFO]    :: Typing files
[INFO]    :: Adding file type information to the fitstbl
[INFO]    :

In [24]:
# The compiled parameter set is in par
print(par)

Parameter     Value      Default    Type          Callable
----------------------------------------------------------
rdx           see below  see below  ParSet, dict  False   
calibrations  see below  see below  ParSet, dict  False   
scienceframe  see below  see below  ParSet, dict  False   
objects       see below  see below  ParSet, dict  False   
extract       see below  see below  ParSet, dict  False   
skysubtract   see below  see below  ParSet, dict  False   
flexure       see below  see below  ParSet, dict  False   
fluxcalib     see below  see below  ParSet, dict  False   

rdx
Parameter     Value                    Default  Type        Callable
--------------------------------------------------------------------
spectrograph  shane_kast_blue          None     str         False   
pipeline      ARMS                     None     str         False   
detnum        None                     None     list        False   
sortroot      shane_kast_blue_setup_A  None     str         

### Directly from a pypit file

Alternatively, you can get the parameter set directly from a pypit file like this:

In [25]:
from pypit.par.util import parse_pypit_file
from pypit.spectrographs.util import load_spectrograph

# Read the PypeIt file
cfg, data, frametype, setups = parse_pypit_file(pypit_file) #,file_check=False)
# Long-winded way of getting the spectrograph name
name = pypitpar.PypitPar.from_cfg_lines(merge_with=cfg)['rdx']['spectrograph']
# Instantiate the spectrograph
spectrograph = load_spectrograph(name)
# Get the spectrograph specific configuration
spec_cfg = spectrograph.default_pypit_par().to_config()
# Initialize the PypeIt parameters merge in the user config
par = pypitpar.PypitPar.from_cfg_lines(cfg_lines=spec_cfg, merge_with=cfg)
print(par)

Parameter     Value      Default    Type          Callable
----------------------------------------------------------
rdx           see below  see below  ParSet, dict  False   
calibrations  see below  see below  ParSet, dict  False   
scienceframe  see below  see below  ParSet, dict  False   
objects       see below  see below  ParSet, dict  False   
extract       see below  see below  ParSet, dict  False   
skysubtract   see below  see below  ParSet, dict  False   
flexure       see below  see below  ParSet, dict  False   
fluxcalib     see below  see below  ParSet, dict  False   

rdx
Parameter     Value                    Default  Type        Callable
--------------------------------------------------------------------
spectrograph  shane_kast_blue          None     str         False   
pipeline      ARMS                     None     str         False   
detnum        None                     None     list        False   
sortroot      shane_kast_blue_setup_A  None     str         

[INFO]    :: Loading the reduction file
[INFO]    :: Found 2 raw data frames
[INFO]    :: Input file loaded successfully


### Directly from a config file

Lastly, you can setup a `PypitPar` object directly using a config file.

In [26]:
p = PypitPar.from_cfg_file(merge_with='keck_lris_blue_long_400_3400_d560.cfg')

I actually had to comment out the definition of the flexure spectrum because I can't find it on my disk, and the declaration above faults when it can't find the file.

With the declaration above, all the default parameters are set, the spectrograph key (`KECK_LRISb`) is used to find the appropriate spectrograph definition file (`../../pypit/config/spectrographs/KECK_LRISb_spectrograph.cfg`), and both are merged with the alterations in the local `keck_lris_blue_long_400_3400_d560.cfg` file.

You can print the compiled parameters using the `to_config` method.

In [27]:
p.to_config('compiled_example.cfg')